In [2]:
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
tf.__version__

'1.8.0'

In [3]:
#mnist datapath
DATAPATH = '/Users/srikanth_m07/Documents/ml_dataset/vision/mnist'

In [11]:
# Define paramaters for the model
learning_rate = 0.01
batch_size = 128
n_epochs = 30

In [5]:
mnist = input_data.read_data_sets(DATAPATH, one_hot=True)
X_batch, Y_batch = mnist.train.next_batch(batch_size)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/srikanth_m07/Documents/ml_dataset/vision/mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/srikanth_m07/Documents/ml_dataset/vision/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /Users/srikanth_m07/Documents/ml_dataset/vision/mnist/t10k-images-idx3-ubyte.gz
Extracting /Users/srikanth_m07/Documents/ml_dataset/vision/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [9]:
X_batch[0].shape, Y_batch[0].shape

((784,), (10,))

In [12]:
#define convenience functions
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [13]:
# Define the cnn
def cnn(X):
    X = tf.reshape(X, [-1, 28, 28, 1])

    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(X, W_conv1) + b_conv1)

    h_pool1 = max_pool_2x2(h_conv1)

    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

    h_pool2 = max_pool_2x2(h_conv2)

    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])
    h_pool2 = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2, W_fc1) + b_fc1)

    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])
    logits = tf.matmul(h_fc1, W_fc2) + b_fc2
    return logits

In [14]:
# Step 2: create placeholders for features and labels
X = tf.placeholder(tf.float32, [batch_size, 784], name='image')
Y = tf.placeholder(tf.float32, [batch_size, 10], name='label')

# Step 3: create weights and bias
w = tf.Variable(tf.zeros([784, 10]), dtype=tf.float32, name='weights')
b = tf.Variable(tf.zeros([batch_size, 10]), dtype=tf.float32, name='biases')

In [16]:
# Step 4: build model
# the model that returns the logits.
logits = cnn(X)

# Step 5: define loss function
entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y)
loss = tf.reduce_mean(entropy)

# Step 6: define training op
# using gradient descent to minimize loss
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [17]:
with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    n_batches = int(mnist.train.num_examples / batch_size)
    for i in range(n_epochs):
        total_loss = 0

        for _ in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            _, loss_batch = sess.run([optimizer, loss], feed_dict={X: X_batch, Y: Y_batch})
            total_loss += loss_batch
        print('Average loss epoch {0}: {1}'.format(i, total_loss / n_batches))

    print('Total time: {0} seconds'.format(time.time() - start_time))
    print('Optimization Finished!')

    # test the model
    preds = tf.nn.softmax(logits)
    correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))  # need numpy.count_nonzero(boolarr) :(

    n_batches = int(mnist.test.num_examples / batch_size)
    total_correct_preds = 0

    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        accuracy_batch = sess.run(accuracy, feed_dict={X: X_batch, Y: Y_batch})
        total_correct_preds += accuracy_batch
        
    print('Accuracy {0}'.format(total_correct_preds / mnist.test.num_examples))

Average loss epoch 0: 1.0204847253997982
Average loss epoch 1: 0.09009565751443863
Average loss epoch 2: 0.07485091570321914
Average loss epoch 3: 0.06305059453251062
Average loss epoch 4: 0.06270117520589147
Average loss epoch 5: 0.05879377319948587
Average loss epoch 6: 0.055868338314286656
Average loss epoch 7: 0.0542993393660434
Average loss epoch 8: 0.0539277875509671
Average loss epoch 9: 0.06109863029557361
Average loss epoch 10: 0.05445739214688741
Average loss epoch 11: 0.05641381915389112
Average loss epoch 12: 0.05212734671584211
Average loss epoch 13: 0.06657804852675284
Average loss epoch 14: 0.055602319488526564
Average loss epoch 15: 0.055434731011179406
Average loss epoch 16: 0.042822760506614495
Average loss epoch 17: 0.06966188763678764
Average loss epoch 18: 0.06721180382466292
Average loss epoch 19: 0.04704909521010524
Average loss epoch 20: 0.04546737606209938
Average loss epoch 21: 0.05504810273067046
Average loss epoch 22: 0.05943649062692944
Average loss epoch 2